<a href="https://colab.research.google.com/github/jasmis1229/task3-1/blob/main/task3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def build_weighted_matrix(tokenized_corpus, word_to_id, window_size=2, method="inverse"):
    vocab_size = len(word_to_id)
    cooccur = defaultdict(float)

    for sentence in tokenized_corpus:
        for i, center in enumerate(sentence):
            if center not in word_to_id:
                continue
            center_id = word_to_id[center]
            for j in range(max(0, i - window_size), min(len(sentence), i + window_size + 1)):
                if i == j or sentence[j] not in word_to_id:
                    continue
                context_id = word_to_id[sentence[j]]
                distance = abs(j - i)

                if method == "inverse":
                    weight = 1.0 / distance
                elif method == "linear":
                    weight = max(0, 1 - distance / window_size)
                elif method == "exponential":
                    weight = np.exp(-distance)
                else:
                    weight = 1.0

                cooccur[(center_id, context_id)] += weight

    row, col, data = zip(*[(i, j, v) for (i, j), v in cooccur.items()])
    return sparse.csr_matrix((data, (row, col)), shape=(vocab_size, vocab_size))
